In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore") # so many settingwithcopywarnings...

## Importing BFA files

In [2]:
dbfa2 = pd.read_csv('../BFA_data/Combined_Counts/dBFA2_counts_with_env_info.csv')
hbfa1 = pd.read_csv('../BFA_data/Combined_Counts/hBFA1_counts_with_env_info.csv')
hbfa2 = pd.read_csv('../BFA_data/Combined_Counts/hBFA2_counts_with_env_info.csv')
nd = {'dBFA2': dbfa2, 'hBFA1': hbfa1, 'hBFA2': hbfa2}

## Reading timepoint exclusion info from file

In [3]:
tp_exclusion = pd.read_excel('accessory_files/tp_exclusion_list.xlsx', sheet_name='exclusion_list')
tp_ex = defaultdict(lambda: defaultdict(lambda: defaultdict(list))) # dict like tp_ex[bfa_name][env_name][replicate] = list of excluded tps
for row in np.array(tp_exclusion[['ASSAY', 'ENV', 'REP', 'TIME']]):
    tp_ex[row[0]][row[1]][row[2]] = str(row[3]).split(';')
print('Example', tp_ex['hBFA1']['FLC4'])

Example defaultdict(<class 'list'>, {'R2': ['16']})


# High AT exclusion

In [4]:
def reverse_complement(seq):
    """reverse complements a dna sequence (does not convert any non-atcg/ATCG characters)"""
    watson_crick = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'a': 't', 't': 'a', 'g': 'c', 'c': 'g'}
    return ''.join([watson_crick.setdefault(c, c) for c in seq[::-1]])

# including the sequence between the barcodes to get the max AT run of the whole region (which is flanked by CG seqs)
middle_seq = 'ATAACTTCGTATAATGTATGCTATACGAAGTTAT'

def gc(s):
    return len([i for i in s if i in ['G', 'C']])
    
def sliding_window_min(row, win_size):
    s = row['Diverse.BC'] + middle_seq + reverse_complement(row['Environment.BC'])
    return min([gc(s[i:i+win_size]) for i in range(len(s)-win_size+1)])

for b in nd:
    # also doing some renaming here
    nd[b] = nd[b][nd[b].apply(lambda r: sliding_window_min(r, 26)>4, axis=1)].rename(columns = {'Full.BC': 'Barcode', 'Subpool.Environment': 'Home_Environment'})

In [5]:
# Getting putative neutral classes
putative_neuts = dict()
putative_neuts['hBFA1'] = list(nd['hBFA1'].loc[nd['hBFA1']['Home_Environment'] == 'YPD_alpha']['Barcode'])
putative_neuts['hBFA2'] = list(nd['hBFA2'].loc[nd['hBFA2']['Home_Environment'] == 'CLM_2N']['Barcode'])
putative_neuts['dBFA2'] = list(nd['dBFA2'].loc[nd['dBFA2']['Home_Environment'] == 'Ancestor_YPD_2N'].loc[nd['dBFA2']['Which.Subpools'] == '-R1-1']['Barcode'])

for n in putative_neuts:
    print(n, len(putative_neuts[n]))

hBFA1 242
hBFA2 298
dBFA2 91


# Simple s measuring code

In [6]:
# Simple s measurement functions
def calc_interval_logslope(row, t1, t2, time1, time2, sum1, sum2, read_cutoff, gens_per_day):
    # simply calculates the slope of log(freq) if boths tps have >=read_cutoff reads (else returns nan)
    if row[t1] < read_cutoff or row[t2] < read_cutoff:
        return np.nan
    else:
        logdif = np.log(row[t2]/sum2) - np.log(row[t1]/sum1)
        return logdif / (gens_per_day*(time2-time1))

def get_s_rec(row, tps, read_cutoff):
    # returns a list of s values from different time intervals for this barcode
    use_tps = [i for i in tps if row[i] >= read_cutoff]
    return [row['scaled.s_' + str(use_tps[i]) + '_' + str(use_tps[i+1])] for i in range(len(use_tps)-1)]

def get_neut_fit(neut_data, t1, t2, time1, time2, sum1, sum2, read_cutoff, gens_per_day):
    # checking if we have enough counts to just use the median log slope (almost always the case)
    usable = neut_data[(neut_data[t1] >= read_cutoff) & (neut_data[t2] >= read_cutoff)]
    if len(usable) >= 10:
        return np.nanmedian(neut_data['s_' + t1 + '_' + t2])
    # combining lineages until we get enough counts to measure neutral fitness (just needed for CLM and FLC)
    # sorting to shuffle barcodes haphazardly
    #print('ok', t1, t2, neut_data[[t1, t2]])
    neut_data = neut_data.sort_values('Barcode')
    # First step, combining to get only 20 barcodes:
    neut_data['grouper'] = [i//(len(neut_data)/20) for i in range(len(neut_data))]
    neut_data = neut_data[['grouper', t1, t2]].groupby('grouper').sum().reset_index()
    # Now keep combining two by two until we get at least 3 barcodes with the minimum read number
    while len(neut_data[(neut_data[t1] >= read_cutoff) & (neut_data[t2] >= read_cutoff)]) < 3:
        neut_data['grouper'] = [i//2 for i in range(len(neut_data))] # combine 2 barcodes together
        neut_data = neut_data[['grouper', t1, t2]].groupby('grouper').sum().reset_index()
        if len(neut_data) < 3:
            #print('akk', neut_data)
            return np.nan
    usable = neut_data[(neut_data[t1] >= read_cutoff) & (neut_data[t2] >= read_cutoff)]
    #print('oh', len(usable), usable[[t1, t2]])
    return np.nanmedian([calc_interval_logslope(r, t1, t2, time1, time2, sum1, sum2, read_cutoff, gens_per_day) for jnk, r in usable.iterrows()])
    

def simple_s_measuring(td, tps, times, neut_bcs, neut_reference_read_cutoff, gens_per_day, read_cutoff=10):
    # calculates log-slopes for all lineages for all possible time intervals, scales by the median log-slope of the neutral class and then for
    # each lineages, using all timepoints with at least read_cutoff reads, averages the consecutive time interval scaled s values to get a lineage s ('s'). 
    # Also records the number of intervals used ('len.s'), and the standard error of the different measures ('stderr.s')
    td['Total.Reads'] = np.sum(td[[t for t in tps]], axis=1)
    for i in range(len(tps)-1):
        for j in range(i+1, len(tps)):
            s1 = sum(td[tps[i]])
            s2 = sum(td[tps[j]])
            td['s_' + str(tps[i]) + '_' + str(tps[j])] = td.apply(lambda r: calc_interval_logslope(r, tps[i], tps[j], times[i], times[j], s1, s2, read_cutoff, gens_per_day), axis=1)
            neut_data = td[(td['Barcode'].isin(neut_bcs)) & (td['Total.Reads'] > neut_reference_read_cutoff)]
            median_neut_fit = get_neut_fit(neut_data, tps[i], tps[j], times[i], times[j], s1, s2, read_cutoff, gens_per_day)
            #median_neut_fit = np.nanmedian(td[(td['Barcode'].isin(neut_bcs)) & (td['Total.Reads'] > neut_reference_read_cutoff)]['s_' + str(tps[i]) + '_' + str(tps[j])])
            td['scaled.s_' + str(tps[i]) + '_' + str(tps[j])] = td['s_' + str(tps[i]) + '_' + str(tps[j])] - median_neut_fit
    # calling get_s_rec 3 times is silly but hey I'm doing it...
    td['s'] = td.apply(lambda r: np.nanmean(get_s_rec(r, tps, read_cutoff)), axis=1) # mean s across timepoint intervals
    td['len.s'] = td.apply(lambda r: len([s for s in get_s_rec(r, tps, read_cutoff) if not np.isnan(s)]), axis=1) # this is number of timepoint intervals
    td['stderr.s'] = td.apply(lambda r: np.nanstd(get_s_rec(r, tps, read_cutoff))/np.sqrt(r['len.s']), axis=1) # this is stderr across timepoint intervals
    return td[['Barcode', 's', 'len.s', 'stderr.s']]

In [7]:
envs_use = ['YPD', 'SC', '37C', 'pH3_8', 'pH7_3', '21C', 'GlyEtOH', 'FLC4', 'CLM', '02M_NaCl']
home_envs_use = [e+'_alpha' for e in envs_use] + [e+'_2N' for e in envs_use] + ['Ancestor_YPD_2N']

simple_fit_data = defaultdict(dict)
c_times = [1, 2, 3, 4, 5]
for bfa_name in nd:
    td = nd[bfa_name][nd[bfa_name]['Home_Environment'].isin(home_envs_use)]
    bcs = list(td['Barcode'])
    for env in envs_use:
        print(bfa_name, env)
        simple_fit_data[bfa_name][env] = dict()
        reps = sorted(set([i.split('-')[2] for i in td.columns if 'Time' in i and env in i]))
        for rep in reps:
            if not 'EXCLUDE ALL' in tp_ex[bfa_name][env][rep]:
                excluded_tps = tp_ex[bfa_name][env][rep]
                tps = [bfa_name + '-' + env + '-' + rep + '-Time' + str(i*8) for i in c_times]
                # to exclude timepoints I will just zero out the counts so they will be caught by the low coverage thresh
                for tp in tps:
                    if tp[tp.index('Time')+4:] in excluded_tps or tp not in td: # also fill in zeros if this tp is not in the data (eg tp 4 not sequenced in hBFA1)
                        td[tp] = np.zeros(len(td))
                included_tps = [t for t in tps if np.sum(td[t]) > 5e4]
                if len(included_tps) < 2:
                    print(bfa_name, env, rep, 'not enough tps')
                else:
                    simple_fit_data[bfa_name][env][rep] = simple_s_measuring(td, tps, c_times, putative_neuts[bfa_name], 30, 8)


dBFA2 YPD
dBFA2 SC
dBFA2 37C
dBFA2 pH3_8
dBFA2 pH7_3
dBFA2 21C
dBFA2 GlyEtOH
dBFA2 FLC4
dBFA2 CLM
dBFA2 02M_NaCl
hBFA1 YPD
hBFA1 SC
hBFA1 37C
hBFA1 pH3_8
hBFA1 pH7_3
hBFA1 21C
hBFA1 GlyEtOH
hBFA1 FLC4
hBFA1 CLM
hBFA1 02M_NaCl
hBFA2 YPD
hBFA2 SC
hBFA2 37C
hBFA2 pH3_8
hBFA2 pH7_3
hBFA2 21C
hBFA2 GlyEtOH
hBFA2 FLC4
hBFA2 CLM
hBFA2 02M_NaCl


# What data do I want?

### For each BFA in each env:
* 1 csv like: Barcode, Test_Environment, Home_Environment, Putative_Neutral, s_R1, s_R2, s_R3, s_ave, sM_R1, sM_R2, sM_R3, sM_ave
* another like: Barcode, Test_Environment, Home_Environment, Putative_Neutral, Replicate, Time, Freq

In [9]:
def inverseVarAve_w_nan(meanVals,standardDevs):
    """
    from https://github.com/barcoding-bfa/fitness-assay-python
    inverseVarAve - take weighted average with inverse variances.
    :param meanVals: Values to be averaged, N x q. Averaged across second dimension
    :param standardDevs: Standard errors of each value, N x q
    :return weightedMeans: N x 1 vector of weighted average
    :return weightedStandardDevs: N x 1 vector of final standard error
    """
    weightedMeans = np.nansum(meanVals*np.power(standardDevs,-2),axis=1)/np.nansum(np.power(standardDevs,-2),axis=1)
    weightedStandardDevs = np.power(np.nansum(np.power(standardDevs,-2),axis=1),-0.5)
    return weightedMeans, weightedStandardDevs

def get_max_error(errors):
    # see comment below - this is to deal with std errs of 0 when there is only one measurement
    if len([i for i in errors if i==0 or pd.isnull(i)])==len(errors):
        return 1
    else:
        return np.nanmax(errors)

fitd = dict()
freqd = dict()
bfa_reps = {'hBFA1': ['R1', 'R2'], 'hBFA2': ['R1', 'R2', 'R3'], 'dBFA2': ['R1', 'R2', 'R3']}
key_cols = ['Barcode', 'Test_Environment', 'Home_Environment', 'Putative_Neutral']
for bfa_name in nd:
    fit_dats = []
    freq_dats = []
    for env in simple_fit_data[bfa_name]:
        freq_dat = nd[bfa_name][nd[bfa_name]['Home_Environment'].isin(home_envs_use)]
        freq_dat['Putative_Neutral'] = freq_dat['Barcode'].isin(putative_neuts[bfa_name])
        freq_dat['Test_Environment'] = [env]*len(freq_dat)
        td = freq_dat[key_cols]
        for rep in bfa_reps[bfa_name]:
            if rep in simple_fit_data[bfa_name][env]:
                b2s = {i[0]:i[1:] for i in np.array(simple_fit_data[bfa_name][env][rep][['Barcode', 's', 'stderr.s']])}
                td['s_'+rep] = td['Barcode'].apply(lambda b: b2s[b][0])
                td['s_'+rep+'_error'] = td['Barcode'].apply(lambda b: b2s[b][1])
            else:
                nanner = [np.nan]*len(td)
                td['s_'+rep], td['s_'+rep+'_error'] = (nanner, nanner)
                
            excluded_tps = tp_ex[bfa_name][env][rep]
            tps = [bfa_name + '-' + env + '-' + rep + '-Time' + str(i*8) for i in c_times]
            tps_use = {tp:int(tp[tp.index('Time')+4:]) for tp in tps if tp[tp.index('Time')+4:] not in excluded_tps and tp in freq_dat}
            for tp in tps_use:
                freq_dat[tps_use[tp]] = np.clip(np.log10(freq_dat[tp]/np.sum(freq_dat[tp])), -6, 0)
            
            ttd = freq_dat[key_cols+list(tps_use.values())].melt(id_vars=key_cols, var_name='Time', value_name='Clipped_Log10(Freq)')
            ttd['Replicate'] = [rep]*len(ttd)
            freq_dats.append(ttd)
        s_aves = np.array(td[['s_'+r for r in bfa_reps[bfa_name]]])
        # In some cases, measurements had only one time interval, so they didn't have an error (std err is 0)
        # in these cases, I will set the error to the max error for the other replicates (and 1 if none of the reps have standard errors)
        error_cols = ['s_'+r+'_error' for r in bfa_reps[bfa_name]]
        td['error_max'] = td.apply(lambda row: get_max_error([row[i] for i in error_cols]), axis=1)
        for i in error_cols:
            td[i] = td.apply(lambda row: np.max([row['error_max'], row[i]]), axis=1)
        s_errs = np.array(td[error_cols])
        td['s_iva'], td['s_iva_err'] = inverseVarAve_w_nan(s_aves, s_errs)
        fit_dats.append(td[[i for i in td if i!='error_max']])
    fitd[bfa_name] = pd.concat(fit_dats)
    freqd[bfa_name] = pd.concat(freq_dats)
    fitd[bfa_name].to_csv('Final_data_sets/'+bfa_name+'_all_fitness.csv', index=False)
    freqd[bfa_name].to_csv('Final_data_sets/'+bfa_name+'_all_freqs_tidy.csv', index=False)

In [13]:
# Scaling haploid fitnesses measured in hBFA correctly
# the reference class used in hBFA2 was CLM_2N, 
# so I will use the median YPD_alpha fitness in hBFA 2 to scale haploid fitnesses

haploid_disadvantage = dict()
td = fitd['hBFA2']
for te in set(td['Test_Environment']):
    haploid_disadvantage[te] = np.nanmedian(td[(td['Home_Environment']=='YPD_alpha') & (td['Test_Environment']==te)]['s_iva'])
    
haploid_disadvantage

def fix_fitness(row):
    # if this is a haploid strain in hBFA2, adjust the fitness to be scaled to the haploid ancestor
    if row['BFA']=='hBFA2' and '_alpha' in row['Home_Environment']:
        return row['s_iva'] - haploid_disadvantage[row['Test_Environment']]
    else:
        return row['s_iva']
    
def fix_put_neut(row):
    if row['BFA'] == 'hBFA2' and row['ploidy'] == 'alpha':
        return row['Home_Environment'] == 'YPD_alpha'
    else:
        return row['Putative_Neutral']
        

env_map = {'GlyEtOH': 'Glycerol/Ethanol', '37C': '37°C', 'pH7_3': 'pH 7.3', 'FLC4': 'Fluconazole', 'CLM': 'Clotrimazole',
          '21C': '21°C', 'SC': 'SC', 'YPD': 'YPD', 'pH3_8': 'pH 3.8', '02M_NaCl': '02M NaCl'}
fds = []
bfas = ['hBFA1', 'hBFA2', 'dBFA2']
for bfa in bfas:
    tmp = fitd[bfa]
    tmp['BFA'] = bfa
    tmp['s'] = tmp.apply(fix_fitness, axis=1)
    tmp['ploidy'] = tmp['Home_Environment'].str.split('_').str[-1]
    tmp['Putative_Neutral'] = tmp.apply(fix_put_neut, axis=1)
    fds.append(tmp[['BFA', 'Barcode', 'Home_Environment', 'ploidy', 'Putative_Neutral', 'Test_Environment', 's']])
    
fd = pd.concat(fds)
fd.to_csv('Final_data_sets/All_fitness_tidy.csv', index=False)
fd

,BFA,Barcode,Home_Environment,ploidy,Putative_Neutral,Test_Environment,s
2,hBFA1,CGCACAAGAAAGAATAATCTTGAATTGGTAAAACCAGATTTGGCAT...,GlyEtOH_alpha,alpha,False,YPD,0.071075
4,hBFA1,AATAAAAGAAGGAAAAGCATTTAAACAAACAAACTTTCTTTTTTCT...,YPD_alpha,alpha,True,YPD,0.093827
5,hBFA1,ATTAAAAAATGAAAGTCTGTTCAATGGAATCAATCAAGTTTCTGTT...,FLC4_alpha,alpha,False,YPD,0.055776
6,hBFA1,TGTATAAAGTAGAAAGAGTTTCACTGAGGAGAACGGGGTTGGGGGT...,GlyEtOH_alpha,alpha,False,YPD,0.062370
7,hBFA1,CGATGAAAGGGAAAATGAATTAACAGTTCCCAAGCGAATTATATTT...,CLM_alpha,alpha,False,YPD,-0.005730
...,...,...,...,...,...,...,...
5856,dBFA2,TTTGCAAACGAGAATGTCTTTCTAAGTTATCAAACTTGTTTTTCGT...,21C_2N,2N,False,02M_NaCl,NaN
5858,dBFA2,CCATTAACCGGTAAAAACGTTTGTATAAGGTAATAGAGTTTTTCTT...,pH7_3_2N,2N,False,02M_NaCl,NaN
5860,dBFA2,CTCAAAATAATAAAACGAGTTTCATCGTGTCAATCGATTTTTTTGT...,02M_NaCl_2N,2N,False,02M_NaCl,NaN
5862,dBFA2,ATAATAACTGACAAATCCTTTCACTCTACCCAACGAAATTGTGTTT...,02M_NaCl_2N,2N,False,02M_NaCl,NaN
